## Verify Cluster Mirror from Java

The purpose of this script is to copy the same methods as in the Javascript equivalent. This will be the main test to determine that the Python integration was successful. 

First, load in some global variables. 

In [1]:
import ee
import GetImageCollection as ic
import CloudCodes as cc
import Subroutines as sub 
import MergeChunk as mc

nhd = ee.FeatureCollection("users/mincej20/NHD_Filtered_addID").sort("AreSqKm", False).distinct(".geo")
gsw = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('occurrence')

We'll also need some visualization parameters: 

In [2]:
viz = {
    'bands': ee.List([ee.String("red"), ee.String("green"), ee.String("blue")]),
    'min': 0, 
    'max': 3000, 
    'gamma': 2.0
}

blueViz = {
    'bands': ['blue'],
    'min': 0, 
    'max': 3000,
    'palette': ['lightblue', 'darkblue']
}

irViz = {
  'bands': ['swir1'], 
  'min': 0, 
  'max': 3000, 
  'gamma': 1.4
}

fadeViz = {
  'bands': ["red", "green", "blue"],
  'min': 0, 
  'max': 3000,
  'gamma': 1.4,
  'opacity': 0.2
}


We're going to need to define some functions necessary for visualization as well. 

In [3]:
def vizColor(img):
    return img.visualize(**viz).copyProperties(img, img.propertyNames())
def vizFaded(img):
    return img.visualize(**fadeViz).copyProperties(img, img.propertyNames())
def vizIr(img):
    return img.visualize(**irViz).copyProperties(img, img.propertyNames())
def waterViz(img):
    return img.visualize(**blueViz).copyProperties(img, img.propertyNames())

def blendGeom(geom):
    def main(img):
        return img.blend(geom).copyProperties(img, img.propertyNames())
    return main

def blendWaterBackground(tovis, geom):
    def main(img):
        ID = img.get("LANDSAT_ID")
        overlay = (ee.Image(tovis.filterMetadata("LANDSAT_ID", "equals", ID)
                            .first()))
        return img.blend(overlay).copyProperties(img, img.propertyNames())
    return main


Lastly, we'll set up the parsing variables for the end conditions in the loop: 

In [4]:
end = 20
offset = 1000

Now to set up the main loop and print the objects for comparing to the Javascript version: 

In [5]:
for i in range(0, end):
    feat = ee.Feature(nhd.toList(end+1, offset).get(i))
    geom = feat.geometry()
    imgs = ic.getImageCollection(feat, False)
    imgs = (imgs.sort("system:time_start")
            .filterMetadata("SATELLITE", "equals", "LANDSAT_5"))
    imgs = ee.ImageCollection(imgs.toList(9))
    
    befIsol = ee.ImageCollection(mc.mergeByChunk(imgs))
    
    aftIsolMNDWI = (imgs.map(sub.noClouds)
                    .map(sub.waterIsolationMNDWI(gsw, geom)))
    aftIsolMNDWI = ee.ImageCollection(mc.mergeByChunk(aftIsolMNDWI))
    
    aftIsolSWIR = (imgs.map(sub.noClouds)
                   .map(sub.waterIsolationSWIR(gsw, geom)))
    aftIsolSWIR = ee.ImageCollection(mc.mergeByChunk(aftIsolSWIR))
    
    template = ee.Image().byte()
    box = geom.bounds()
    outline = (template.paint(ee.FeatureCollection(feat), "red", 2)
               .visualize(**{"palette": "red"}))
    
    background = befIsol.map(vizFaded)
    backBlend = background.map(blendGeom(outline))
    
    befColorImgs = befIsol.map(vizColor)
    befBlendImgs = befColorImgs.map(blendGeom(outline))
    
    irColorImgs = befIsol.map(vizIr)
    irBlendImgs = irColorImgs.map(blendGeom(outline))
    
    fadeColorImgsMNDWI = aftIsolMNDWI.map(vizColor)
    fadeBlendImgsMNDWI = backBlend.map(blendWaterBackground(fadeColorImgsMNDWI, geom))
    
    fadeColorImgsSWIR = aftIsolSWIR.map(vizColor)
    fadeBlendImgsSWIR = backBlend.map(blendWaterBackground(fadeColorImgsSWIR, geom))
    
    aftColorImgsMNDWI = aftIsolMNDWI.map(waterViz)
    aftBlendImgsMNDWI = backBlend.map(blendWaterBackground(aftColorImgsMNDWI, geom))
    aftBlendImgsMNDWI = aftBlendImgsMNDWI.map(blendGeom(outline))
    
    aftColorImgsSWIR = aftIsolSWIR.map(waterViz)
    aftBlendImgsSWIR = backBlend.map(blendWaterBackground(aftColorImgsSWIR, geom))
    aftBlendImgsSWIR = aftBlendImgsSWIR.map(blendGeom(outline))
    
    blendImgs = (befBlendImgs.merge(irBlendImgs)
                 .merge(fadeBlendImgsSWIR)
                 .merge(aftBlendImgsSWIR)
                 .merge(fadeBlendImgsMNDWI)
                 .merge(aftBlendImgsMNDWI)
                 .sort("system:time_start")
                )
    
    filmArgs = {
        'region': box, 
        'dimensions': 700, 

    }
    print(feat.get("NHDPlID").getInfo())
    print(blendImgs.getFilmstripThumbURL(filmArgs))


50000200171540
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/filmstripThumbnails/8b0bc724cd67bb4565f97d3be698bcfe-27e8fb29f35abd25ee2df02cde0dee8a:getPixels
5000900047103
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/filmstripThumbnails/5e89328fef3b6cdb2288d5635610734b-374ff6d0a83833efd9a3ebf98793231d:getPixels
20000500201612
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/filmstripThumbnails/fb15804fd26b3e2b0e82a6bba32a3304-dc80e20cb635c29edc0a21bcf32ab785:getPixels
65000300115334
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/filmstripThumbnails/e9013fad78e609074b20feee316a9973-e9a8f97e587d1c7f7545146ed11479d8:getPixels
55001200131526
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/filmstripThumbnails/472af88c412ffe3bad71958c051dc979-8d7d05dbb82ec1f4db20e5e9d1d4901e:getPixels
24000300092727
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/films

In [5]:
for i in range(0, end):
    feat = ee.Feature(nhd.toList(end+1, offset).get(i))
    geom = feat.geometry()
    imgs = ic.getImageCollection(feat, False)
    befChunk5 = (imgs.sort("system:time_start")
                 .filterMetadata("Chunk", "equals", 16)
                 .filterMetadata("Year", "equals", 1990)
                 .filterMetadata("SATELLITE", "equals", "LANDSAT_5"))
    befChunk7 = (imgs.sort("system:time_start")
                 .filterMetadata("Chunk", "equals", 16)
                 .filterMetadata("Year", "equals", 2000)
                 .filterMetadata("SATELLITE", "equals", "LANDSAT_7"))
    befChunk8 = (imgs.sort("system:time_start")
                 .filterMetadata("Chunk", "equals", 16)
                 .filterMetadata("Year", "equals", 2015)
                 .filterMetadata("SATELLITE", "equals", "LANDSAT_8"))
   
    aftChunk5 = ee.ImageCollection(mc.mergeByChunk(befChunk5))
    aftChunk7 = ee.ImageCollection(mc.mergeByChunk(befChunk7))
    aftChunk8 = ee.ImageCollection(mc.mergeByChunk(befChunk8))
    
    template = ee.Image().byte()
    box = geom.bounds()
    outline = (template.paint(ee.FeatureCollection(feat), "red", 2)
               .visualize(**{"palette": "red"}))
    
    bef5 = befChunk5.map(vizColor)
    aft5 = bef5.map(blendGeom(outline))
    bef7 = befChunk7.map(vizColor)
    aft7 = bef7.map(blendGeom(outline))
    bef8 = befChunk8.map(vizColor)
    aft8 = bef8.map(blendGeom(outline))
    
    
    blendImgs = (bef5.merge(aft5)
                 .merge(bef7)
                 .merge(aft7)
                 .merge(bef8)
                 .merge(aft8)
                 .sort("system:time_start")
                )
    
    filmArgs = {
        'region': box, 
        'dimensions': 700, 

    }
    print(feat.get("NHDPlID").getInfo())
    print(blendImgs.getFilmstripThumbURL(filmArgs))

50000200171540
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/filmstripThumbnails/6b6d5d8bc6e9a5cf6339c928f0e1d965-1e4d0a9e35a2e7d032c1da67400c66d4:getPixels
5000900047103
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/filmstripThumbnails/97e59dc2d3f91fe8bc19dafd01b3d663-3f01b714020b6f7565a83686e79b5b97:getPixels
20000500201612
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/filmstripThumbnails/45454a0303b1b021a2b92b0b24604342-f16baf689048d99ecd4e080ec1613310:getPixels
65000300115334
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/filmstripThumbnails/a80ebe3ced4722d1613d4a456a3e270a-d2f3319126ea28f6680a66c57abf919b:getPixels
55001200131526
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/filmstripThumbnails/d25258bdfd837caea3b7845dc35ced58-1058447306f6004d372ac32e9b5edc9d:getPixels
24000300092727
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/films